## Data import

In [ ]:
import pandas as pd
from tqdm import tqdm
import os

import re
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_classes = pd.read_csv('/content/drive/MyDrive/HSE_project/events_with_KNY.csv')
df_classes_new =  pd.DataFrame()
df_classes

,Опасные природные процессы,Описание явления простыми словами,Простые словосочетания для явления,Простые словосочетания для триггеров или последствий,Потенциальное неразличение авторами новостей внутри группы явлений
0,Абразия и термоабразия,Размыв и отступание берега моря. Обычно происх...,отступание берега; разрушение берега; берегова...,после шторма; высокие волны; *объект* упал в в...,А
1,"Переработка берегов водохранилищ, озер","Изменение линии берега, выполняемое другими пр...",отступание берега; разрушение берега; берегова...,*объект* упал в воду; *объект* пришлось перено...,А
2,Карст,"Некоторые горные породы растворяются водой, ка...",появилась дыра; появилась яма; появился провал...,Провалился под землю; просел грунт; продолжите...,Б
3,Суффозия,"В горных породах, состоящих из больших и мален...",появилась дыра; появилась яма; появился провал...,Провалился под землю; просел грунт; продолжите...,Б
4,Просадочность лессовых пород,"Лессовая горная порода, при смачивании, может ...","Просадка, лесс просел, провал поверхности,",Провалился под землю; просел грунт; продолжите...,Б
5,"Подтопление территории, переувлажнение почвы",Когда вода не может уходит в землю и/или утека...,Затопило; стоит вода; половодье; паводок; подн...,Прошел ливень; стаивает снег; переувлажнение п...,С
6,Эрозия плоскостная и овражная,Вынос (смыв) частичек грунта с поверхности вод...,Вырос овраг; край оврага обрушился; дорогу раз...,Смыло плодородный слой; *объект* на краю овраг...,Д
7,Русловые деформации,"Берега, острова и судовые фарватеры некоторых ...",Изменился фарватер; мель; плес; меандры; перек...,Уровень воды снизился; размыло берег; сильный ...,Д
8,Термоэрозия овражная,"Как эрозия, только в условиях мерзлоты. При та...",Вырос овраг; край оврага обрушился; дорогу раз...,тает мерзлота;,Д
9,"Термокарст, деградация мерзлоты",При протаивании мерзлоты (т.е. вытаивании част...,"протаивание подземного льда, вытаивание ледяно...",тает мерзлота; после строительства; деформиров...,Б


In [ ]:
df_classes_new[['classes','description']] = df_classes[['Опасные природные процессы','Описание явления простыми словами']]
classes_list = ';'.join(df_classes_new['classes'].tolist())
classes_list = [f"{row[1]}: {row[2]}" for row in df_classes_new.itertuples()]

paths = ['/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_chechnya',
         '/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_dgstn',
         '/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_ingnsh',
         '/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_kbr',
         '/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_kcr',
         '/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_rso-alania',
         '/content/drive/MyDrive/HSE_project/СКФО/all_publics_SKFO/public_data_stvr',
          ]
objects = []
for path in paths:
    objects.extend(os.listdir(path))
objects

['emergency_20_ramzan.csv',
 'emergency_20_chr.csv',
 'emergency_20_nauradmchr.csv',
 'emergency_20_urusadm.csv',
 'emergency_20_merargun.csv',
 'emergency_20_public219691536.csv',
 'emergency_20_public219650764.csv',
 'emergency_20_grozadm.csv',
 'emergency_20_gudad.csv',
 'emergency_20_public217042926.csv',
 'emergency_20_spitum.csv',
 'emergency_20_akmr95.csv',
 'emergency_20_nozhayurt.csv',
 'emergency_20_public219610386.csv',
 'emergency_20_public217777720.csv',
 'emergency_20_shaligl.csv',
 'emergency_20_shelkadm.csv',
 'emergency_20_club222539909.csv',
 'emergency_05_respublica_dagestan.csv',
 'emergency_05_s.a.melikov.csv',
 'emergency_05_vesti_novolak.csv',
 'emergency_05_ognidagestanskie.csv',
 'emergency_05_izbadm.csv',
 'emergency_05_public205075193.csv',
 'emergency_05_public209064868.csv',
 'emergency_05_gorodkizlyar.csv',
 'emergency_05_go.hasavyurt.csv',
 'emergency_05_goyuzhnosukhokumsk.csv',
 'emergency_05_public205278396.csv',
 'emergency_05_public215536775.csv',
 'e

In [ ]:
dfs = []
for path in paths:
    # Получаем список CSV файлов в текущей папке
    csv_files = [
        f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith(".csv")
    ]

    # Читаем каждый CSV файл и добавляем его в список DataFrame
    for f in csv_files:
        file_path = os.path.join(path, f)
        # Попробуем прочитать файл с разными разделителями
        try:
            df = pd.read_csv(file_path)
        except pd.errors.ParserError:
            try:
                df = pd.read_csv(file_path, sep=';')
            except pd.errors.ParserError:
                try:
                    df = pd.read_csv(file_path, sep=';', on_bad_lines='skip')
                except pd.errors.ParserError:
                    print(f"Error reading {file_path}. File will be skipped.")
                    continue
        dfs.append(df)

# Объединяем все DataFrame в один
df = pd.concat(dfs, ignore_index=True)

<ipython-input-16-e32e798b2d0d>:17: DtypeWarning: Columns (6,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep=';')


In [ ]:
df

,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ПОДПИСЧИКОВ/ДРУЗЕЙ,ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ,АВТОР ЗАПИСИ,ССЫЛКА НА АВТОРА ПОСТА,ПОЛ АВТОРА,ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА,ДАТА ПУБЛИКАЦИИ,ТЕКСТ,ИЛЛЮСТРАЦИИ,ЛАЙКОВ,РЕПОСТОВ,КОММЕНТАРИЕВ,ПРОСМОТРОВ,ИСТОЧНИК,НАЗВАНИЕ ИСТОЧНИКА,ССЫЛКА НА ВИДЕО,ДОКУМЕНТЫ
0,https://vk.com/wall-106849164_174092,Город Москва,307102,https://vk.com/club106849164,Город Москва,https://vk.com/club106849164,NaN,https://vk.com/club106849164?w=wall-106849164_...,2024-07-12 13:30,🎭 Спектакль на свежем воздухе или импровизация...,NaN,9,2,0,876,NaN,NaN,https://vk.com/video-106849164_456242883,NaN
1,https://vk.com/wall-106849164_174088,Город Москва,307102,https://vk.com/club106849164,Город Москва,https://vk.com/club106849164,NaN,https://vk.com/club106849164?w=wall-106849164_...,2024-07-12 12:30,🚂 Посмотреть парад исторических автомобилей и...,https://sun9-8.userapi.com/impg/gavIC5fOMnmche...,10,1,1,927,NaN,NaN,NaN,NaN
2,https://vk.com/wall-106849164_174086,Город Москва,307102,https://vk.com/club106849164,Город Москва,https://vk.com/club106849164,NaN,https://vk.com/club106849164?w=wall-106849164_...,2024-07-12 11:40,🕺Уже в эти выходные гастрономические улицы в ц...,https://sun9-32.userapi.com/impg/_-vVzWeGcCONB...,12,4,1,1254,NaN,NaN,NaN,NaN
3,https://vk.com/wall-106849164_174083,Город Москва,307102,https://vk.com/club106849164,Город Москва,https://vk.com/club106849164,NaN,https://vk.com/club106849164?w=wall-106849164_...,2024-07-12 10:40,«Техноград» приглашает на курсы по кибербезопа...,https://sun9-5.userapi.com/impg/KsZbD2_CYq9u3Y...,8,1,0,1297,NaN,NaN,NaN,NaN
4,https://vk.com/wall-106849164_174082,Город Москва,307102,https://vk.com/club106849164,Город Москва,https://vk.com/club106849164,NaN,https://vk.com/club106849164?w=wall-106849164_...,2024-07-12 09:40,🏡 Фестиваль «Усадьбы Москвы» продолжается. С 1...,https://sun9-20.userapi.com/impg/4dZeI5xS9yxTh...,25,7,1,6548,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126182,https://vk.com/wall-169705869_10,Москва • Новости,49130,https://vk.com/club169705869,Москва • Новости,https://vk.com/club169705869,NaN,https://vk.com/club169705869?w=wall-169705869_10,2018-08-04 22:15,В тройку лучших московских водителей автобуса ...,https://sun9-38.userapi.com/s/v1/if1/zzCqHxW8G...,0,0,0,43,NaN,NaN,NaN,NaN
126183,https://vk.com/wall-169705869_9,Москва • Новости,49130,https://vk.com/club169705869,Москва • Новости,https://vk.com/club169705869,NaN,https://vk.com/club169705869?w=wall-169705869_9,2018-08-04 21:21,Комнаты прохлады: как москвичам помогают спаст...,https://sun9-24.userapi.com/s/v1/if1/l6Ezv8e-y...,0,0,0,6,NaN,NaN,NaN,NaN
126184,https://vk.com/wall-169705869_8,Москва • Новости,49130,https://vk.com/club169705869,Москва • Новости,https://vk.com/club169705869,NaN,https://vk.com/club169705869?w=wall-169705869_8,2018-08-04 20:32,Центральные улицы Москвы перекроют на время но...,https://sun9-21.userapi.com/s/v1/if1/cL89WZVPe...,0,0,0,21,NaN,NaN,NaN,NaN
126185,https://vk.com/wall-169705869_7,Москва • Новости,49130,https://vk.com/club169705869,Москва • Новости,https://vk.com/club169705869,NaN,https://vk.com/club169705869?w=wall-169705869_7,2018-08-04 19:00,На юге Москвы неизвестный отобрал у мужчины рю...,https://sun9-60.userapi.com/s/v1/if1/3YenQ3nv8...,1,1,0,6,NaN,NaN,NaN,NaN


## Filtering by key word

In [ ]:
natural_disaster_words = []
with open('natural_disaster_words.txt', 'r', encoding='utf-8') as file:
  for line in file:
    natural_disaster_words.append(line.strip())

In [ ]:
def normalize_text(text):
  text = str(text)
  # Приведение к нижнему регистру
  text = text.lower()

  # Удаление знаков препинания
  text = re.sub(r'[^\w\s]', '', text)

  # Удаление стоп-слов
  words = word_tokenize(text)
  stop_words = set(stopwords.words('russian'))
  words = [word for word in words if word not in stop_words]
  text = ' '.join(words)

  stemmer = SnowballStemmer("russian")
  words = word_tokenize(text)
  stemmed_words = [stemmer.stem(word) for word in words]
  normalized_text = ' '.join(stemmed_words)

  return normalized_text

# Применяем нормализацию к списку ключевых слов (по словосочетаниям)
normalized_keywords = []
for keywords in natural_disaster_words:
    normalized_keywords.append(normalize_text(keywords))

# Выводим нормализованный список ключевых слов
print(normalized_keywords)

['отступан берег', 'разрушен берег', 'берегов эроз', 'размыван берег', 'шторм', 'высок волн', 'упа вод', 'разруш', 'смыл', 'берег мор', 'отступан берег', 'разрушен берег', 'берегов эроз', 'размыван берег', 'лин вод отступ', 'берег сошел оползен', 'абраз', 'упа вод', 'пришл перенос', 'судн причал', 'появ пляж', 'исчез пляж', 'берег озер', 'берег водохранилищ', 'появ дыр', 'появ ям', 'появ прова', 'просел част', 'провал земл', 'подземн вод разм земл', 'прова', 'воронк', 'пещер', 'полост провал земл', 'просел грунт', 'продолжительн ливн', 'сильн ливн', 'появ дыр', 'появ ям', 'появ прова', 'просел част', 'объект провал земл', 'подземн вод разм земл', 'провал земл', 'просел грунт', 'продолжительн ливн', 'сильн ливн', 'просадк', 'лесс просел', 'прова поверхн', 'провал земл', 'просел грунт', 'продолжительн ливн', 'сильн ливн', 'затоп', 'сто вод', 'половод', 'паводок', 'подня грунтов вод', 'прошел ливен', 'стаива снег', 'переувлажнен почв', 'погибл культур', 'погибл посев площад', 'вырос овраг

In [ ]:
def normalize_description(text):
    text  = str(text)
    # Приведение к нижнему регистру
    text = text.lower()

    # Удаление знаков препинания
    text = re.sub(r'[^\w\s]', '', text)

    # Разбиение на слова
    words = word_tokenize(text)

    # Удаление стоп-слов
    stop_words = set(stopwords.words('russian'))
    words = [word for word in words if word not in stop_words]

    stemmer = SnowballStemmer("russian")
    words = [stemmer.stem(word) for word in words]

    return words

def check_situation(text):
    words_found = []
    count = 0
    for word in natural_disaster_words:
        if word in text:
            words_found.append(word)
            count += 1
    return words_found, count

df['text'] = df['ТЕКСТ']
df['update_text'] = df['text'].apply(normalize_text)
df['words_found'], df['count'] = zip(*df['update_text'].apply(check_situation))

In [ ]:
df = df[df['count'] > 1]
df_description = pd.DataFrame()
df_description = df['text']
df_description

261       🌧️ В ночь на 20 июня в Москве ухудшится погода...
714       🥶 В ночь с 11 на 12 мая в столице ожидаются за...
1096      ❗️В ночь на 5 апреля в Москве ожидаются заморо...
1183      ❗️В Москве сегодня ночью ожидается гололедица....
1276      🏥 В Москве открылись первые детские эндоскопич...
                                ...                        
125149    «Есть надежда на то, что к началу октября иниц...
125180    Ранее МЧС распространило штормовое предупрежде...
125208    В Московском регионе из-за шквалистого ветра д...
125280    С приходом антициклона теплый день с июльскими...
125485    В понедельник, 17 сентября, усилится влияние а...
Name: text, Length: 1643, dtype: object

## Data classify with model

In [ ]:
import requests
import logging
import time
import pandas as pd

time_start = time.time()

logging.basicConfig(filename='api_calls.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Список ваших API ключей]
api_keys = [API_KEY_1, API_KEY_2, API_KEY_3, API_KEY_4, API_KEY_5]
current_key_index = 0


API_ENDPOINT_BASE = "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent"

headers = {"Content-Type": "application/json"}

def send_question_and_get_answer(question):
    global current_key_index
    API_KEY = api_keys[current_key_index]
    API_ENDPOINT = f"{API_ENDPOINT_BASE}?key={API_KEY}"
    data = {
        "contents": [
            {
                "parts": [
                    {
                        "text": question
                    }
                ]
            }
        ]
    }

    try:
        response = requests.post(API_ENDPOINT, json=data, headers=headers)
        response.raise_for_status()
        return response.json()

    except requests.exceptions.RequestException as e:
        logging.error(f"Ошибка при отправке запроса к API: {e}")
        # Добавьте обработку исключений, например, повторную попытку или пропуск запроса
        return None

# Предположим, что df - ваш DataFrame

texts = []
classes = []

for i in range(0, 1642):
    question = (
        f"Определите, относится ли данная запись из социальной сети к новости о природном "
        f"катаклизме, включая обсуждения последствий или подведения итогов и ликвидации последствий, а также о предупреждениях. "
        f"учитывай, что снегопад и морозы говорят, об одном из природных явлений"
        f"Если да, то укажите ВСЕ классы катаклизмов из списка, которые упомянуты в тексте. "
        f"Учитывайте, что заморозки и пожарная опасность являются взаимоисключающими. \n\n"
        f"Если нет, напишите 'нет'. \n\n"
        f"Текст: {df['text'].iloc[i]} \n\n"
        f"Классы катаклизмов: \n\n {classes_list}.  \n\n"
    )

    api_response = send_question_and_get_answer(question)

    if api_response and 'candidates' in api_response and len(api_response['candidates']) > 0:
        if 'content' in api_response['candidates'][0]:
            final_answer_text = api_response['candidates'][0]['content']['parts'][0]['text']
            print("Итоговый ответ:", final_answer_text)
            print("Исходный текст:", df['text'].iloc[i])

            texts.append(df['text'].iloc[i])
            classes.append(final_answer_text)

        else:
            print(f"Ошибка: ключ 'content' отсутствует в ответе API для i={i}")
            texts.append(df['text'].iloc[i])
            classes.append("Ошибка API")
    else:
        print(f"Ошибка: не получен корректный ответ API для i={i}")
        texts.append(df['text'].iloc[i])
        classes.append("Ошибка API")

    # Переключение на следующий API ключ
    current_key_index = (current_key_index + 1) % len(api_keys)

    # Добавьте паузу, чтобы избежать превышения лимитов API
    time.sleep(1)
    print('i=', i)

time_end = time.time()
print("Прошло времени", time_end - time_start)

# Создание DataFrame после завершения цикла
results_df = pd.DataFrame({'text': texts, 'class': classes})
results_df.to_csv('/content/drive/MyDrive/HSE_project/ЦФО/preprocessing_data/26072024_Moscow_test_results_0_1642.csv')

In [ ]:
results_df = pd.DataFrame({'text': texts, 'class': classes})
path_to_save = ''
results_df.to_csv(path_to_save)
df_cleaned = results_df.loc[~results_df['class'].isin(['нет', 'Нет',' нет','нет '])]

In [ ]:
df_cleaned

,text,class
7,Чеченскую Республику накрыли сильные ливни и ш...,Да\n\nКлассы катаклизмов: \n\n* Сильные атмосф...
8,"В связи с продолжительными ливневыми осадками,...","Да, это новость о природном катаклизме, включа..."
10,Очередной борт с гуманитарной помощью для бедс...,"Да,\n\nКлассы катаклизмов, упомянутые в тексте..."
42,Ассаламу алайкум! Прошедшей ночью в результате...,Да\n\nКлассы катаклизмов:\n\n* Наводнение\n* С...
44,Дорогие друзья! По завершении заседания Операт...,да\n\nКлассы катаклизмов:\n\n- Сильные атмосфе...
...,...,...
1653,ШТОРМОВОЕ ПРЕДУПРЕЖДЕНИЕ По сведениям Центра...,Да\n\nКлассы катаклизмов:\n\n* Сильные атмосфе...
1655,По сведениям Калининградского центра по гидром...,"Да\n\nКлассы катаклизмов:\n\n* Ураганы, смерчи..."
1656,МЧС объявило штормовое предупреждение в Калини...,"Да, новость относится к категории предупрежден..."
1660,Уважаемые жители и гости города Светлогорска! ...,"Да, это относится к новости о природном катакл..."


In [ ]:
df_cleaned['Найденные ключевые слова'] = ""

def find_keywords(text):
  keywords_found = []
  for keyword in df_classes['Опасные природные процессы']:
    if keyword.lower() in text.lower():
      keywords_found.append(keyword)
  return ', '.join(keywords_found)

# Преобразование столбца 'class' к строкам
df_cleaned['class'] = df_cleaned['class'].astype(str)

# Применение функции к каждой строке в столбце 'class'
df_cleaned['class'] = df_cleaned['class'].apply(find_keywords)
df_cleaned = df_cleaned.dropna()
df_cleaned.to_csv('/content/drive/MyDrive/HSE_project/ЦФО/final_data/26072024_plan_minimum_Moscow.csv')
df_cleaned

,class,text_x,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ДАТА ПУБЛИКАЦИИ,ТЕКСТ,Найденные ключевые слова
1,Заморозки,🥶 В ночь с 11 на 12 мая в столице ожидаются за...,https://vk.com/wall-106849164_170339,Город Москва,2024-05-11 14:25,🥶 В ночь с 11 на 12 мая в столице ожидаются за...,
2,Заморозки,❗️В ночь на 5 апреля в Москве ожидаются заморо...,https://vk.com/wall-106849164_167671,Город Москва,2024-04-04 12:30,❗️В ночь на 5 апреля в Москве ожидаются заморо...,
3,Гололедно-изморозевые явления,❗️В Москве сегодня ночью ожидается гололедица....,https://vk.com/wall-106849164_167003,Город Москва,2024-03-25 16:40,❗️В Москве сегодня ночью ожидается гололедица....,
5,Гололедно-изморозевые явления,🌧 С 18:00 в Москве ожидается ледяной дождь. На...,https://vk.com/wall-106849164_165027,Город Москва,2024-02-25 14:23,🌧 С 18:00 в Москве ожидается ледяной дождь. На...,
6,Сильные атмосферные осадки,"🌨 В субботу в Москве ожидается снегопад, котор...",https://vk.com/wall-106849164_164558,Город Москва,2024-02-17 17:04,"🌨 В субботу в Москве ожидается снегопад, котор...",
...,...,...,...,...,...,...,...
1622,"Заморозки, Сильные атмосферные осадки","В Москве автомобилистов из-за снегопада, котор...",https://vk.com/wall-169705869_3642,Москва • Новости,2018-11-15 06:57,"В Москве автомобилистов из-за снегопада, котор...",
1625,"Наледеобразование, Заморозки",Выборочная противогололедная обработка дорог б...,https://vk.com/wall-169705869_2847,Москва • Новости,2018-10-30 00:08,Выборочная противогололедная обработка дорог б...,
1627,Сильные атмосферные осадки,"Снегопад, начавшийся в Москве вслед за дождями...",https://vk.com/wall-169705869_2782,Москва • Новости,2018-10-28 12:04,"Снегопад, начавшийся в Москве вслед за дождями...",
1640,"Ураганы, смерчи, сильный ветер",В Московском регионе из-за шквалистого ветра д...,https://vk.com/wall-169705869_985,Москва • Новости,2018-09-22 16:26,В Московском регионе из-за шквалистого ветра д...,


In [ ]:
path_all_data = ''
df_all = pd.read_csv(path_all_data)

df_cleaned = df_cleaned.reset_index()

# Добавляем индекс к df2
df_all = df_all.reset_index()

# Объединяем таблицы по новому индексу
merged_df = pd.merge(df_cleaned, df_all, on='index', how='outer')

# Удаляем ненужный столбец 'index'
merged_df = merged_df.drop('index', axis=1)

merged_df = merged_df[['class','text_x','ССЫЛКА НА ПОСТ','НАЗВАНИЕ ВЛАДЕЛЬЦА','ДАТА ПУБЛИКАЦИИ','ТЕКСТ']]
merged_df

,class,text_x,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ДАТА ПУБЛИКАЦИИ,ТЕКСТ
0,нет,🌧️ В ночь на 20 июня в Москве ухудшится погода...,https://vk.com/wall-106849164_172694,Город Москва,2024-06-19 12:30,🌧️ В ночь на 20 июня в Москве ухудшится погода...
1,Да\n\nКлассы катаклизмов:\n\n* Заморозки,🥶 В ночь с 11 на 12 мая в столице ожидаются за...,https://vk.com/wall-106849164_170339,Город Москва,2024-05-11 14:25,🥶 В ночь с 11 на 12 мая в столице ожидаются за...
2,"Да, это новость о природном катаклизме.\n\nКла...",❗️В ночь на 5 апреля в Москве ожидаются заморо...,https://vk.com/wall-106849164_167671,Город Москва,2024-04-04 12:30,❗️В ночь на 5 апреля в Москве ожидаются заморо...
3,Да\n\nКласс катаклизмов: Гололедно-изморозевые...,❗️В Москве сегодня ночью ожидается гололедица....,https://vk.com/wall-106849164_167003,Город Москва,2024-03-25 16:40,❗️В Москве сегодня ночью ожидается гололедица....
4,нет,🏥 В Москве открылись первые детские эндоскопич...,https://vk.com/wall-106849164_166045,Город Москва,2024-03-14 18:30,🏥 В Москве открылись первые детские эндоскопич...
...,...,...,...,...,...,...
1638,нет,"«Есть надежда на то, что к началу октября иниц...",https://vk.com/wall-169705869_1044,Москва • Новости,2018-09-24 03:43,"«Есть надежда на то, что к началу октября иниц..."
1639,нет,Ранее МЧС распространило штормовое предупрежде...,https://vk.com/wall-169705869_1013,Москва • Новости,2018-09-23 11:23,Ранее МЧС распространило штормовое предупрежде...
1640,"Да\n\nКлассы катаклизмов:\n\n* Ураганы, смерчи...",В Московском регионе из-за шквалистого ветра д...,https://vk.com/wall-169705869_985,Москва • Новости,2018-09-22 16:26,В Московском регионе из-за шквалистого ветра д...
1641,нет,С приходом антициклона теплый день с июльскими...,https://vk.com/wall-169705869_913,Москва • Новости,2018-09-21 10:35,С приходом антициклона теплый день с июльскими...
